This notebook will be used for the purpose of preparing a corpus for either tf-idf or doc2vec embedding.
The corpus will be a list of lists, with the doc_id being the first fild in each list of 2.
This will then be transformed for doc2vec embedding.

Punctuation removal, case change, stop word removal, and stemming have already been done as a part of this process.

The corpus will need to include both the historical and current documents, as vectors created will be used to cluster the documents using kmeans.

Future versions of these notebooks will replace the local storage of the binary files with cloud storage.

In [17]:
# This cell will accumulate a list of urls from legiscan that will be used for the text extraction
#from google.cloud import storage
import json
import pickle
import requests
import string
import html2text as h2t
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.tag import pos_tag
from nltk import PorterStemmer
#from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
# create storage client
storage_client = storage.Client.from_service_account_json('cedc-erc-legislation-project-10835c838715.json')

# get bucket with name
bucket = storage_client.get_bucket('cedc-erc-storage')

# get bucket data as blob
#max results restricted to 5 for testing
bloblist = bucket.list_blobs(prefix='LegData/NH')

NameError: name 'storage' is not defined

In [6]:
#outfile = open('2018legcorpus.txt','w')
legtext_corpus = [[0,0,['dummy','entry']]]
textLinetoStartAfter = 'In the Year of Our Lord'
for bl in bloblist:
    if '.json' in bl.name and '/bill/' in bl.name and '2015' not in bl.name and '2016' not in bl.name:
        json_raw = bl.download_as_string().decode("utf-8")
        json_data = json.loads(json_raw)
        #print("Working on: " + bl.name)
        if 'bill_id' in json_data['bill']:
            bill_id = str(json_data['bill']['bill_id'])
            for s in json_data['bill']['texts']:
                state_link = str(s['state_link'])
                doc_id = str(s['doc_id'])
#---in a future version, the doc id will be used to retrieve the text from google cloud as opposed to directly from the state website-------
                #if the bill text is in plain text, pull out just the text
                if 'html' in state_link:
                    try:
                        rawbillText = h2t.html2text(requests.get(state_link).content.decode("utf-8"))
                        i=False
                        billText=' '
                        for l in rawbillText.split('\n'):
                            if i:
                                billText=billText+str(l)+' '
                            if textLinetoStartAfter in l:
                                i=True
                    #else:
                        #otherwise, we need to stream in the file and convert to text via tika
                        #file_size = str(s['text_size'])
                        #file = requests.get(state_link, stream=True)
                        #----------------------------------------------------
                        #currently using tika which appears to be some kind of python wrapper for a java app
                        #will want to replace with something more universal (PDFMiner, perhaps)
                        #raw = parser.from_buffer(file.content)
                        #billText = raw['content']
                        #----------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------------------
                        #tokenize the text
                        tokens = word_tokenize(billText)
                        #create a list of parts of speech tags for the tokens
                        tags = pos_tag(tokens)
                        # convert to lower case
                        tokens = [w.lower() for w in tokens]

                        # remove punctuation
                        table = str.maketrans('', '', string.punctuation)
                        strippedBillText = [w.translate(table) for w in tokens]

                        # remove remaining tokens that are not alphabetic
                        wordsInBill = [word for word in strippedBillText if word.isalpha()]

                        #define the single character 'words' that need to be removed
                        singleCharacters = ['a','b','c','d','e','f','g','h','i','j',\
                                            'k','l','m','n','o','p','q','r','s','t',\
                                            'u','v','w','x','y','z']

                        #grab the list of words that were determined to be proper nouns
                        #ProperNouns = [word.lower() for word, tag in tags if tag == 'NNP' or tag == 'NNPS' or tag == 'IN']

                        #define stop words
                        stop_words = set(stopwords.words('english'))
                        # filter out stop words, single letter words, and proper nouns
                        wordsInBill = [w for w in wordsInBill if not w in stop_words]
                        wordsInBill = [w for w in wordsInBill if not w in singleCharacters]
                        #wordsInBill = [w for w in wordsInBill if not w in ProperNouns]

                        #roll words up to simpler, more categorical forms to make comparison easier
                        # stemming of words
                        #porter = PorterStemmer()
                        #wordsInBill = [porter.stem(word) for word in wordsInBill]

                        #create the corpus object
                        legdoc = [bill_id,doc_id,wordsInBill]
                        legtext_corpus.append(legdoc)
                        #legtext_corpus_ex.extend(legdoc)
                        #print(legtext_corpus)
                        #print(str(bill_id),',',str(doc_id),',',str(wordsInBill),file=outfile)
                        #outfile.write(bill_id)
                        #create a vector based on the tf-idf algorithm (may be better for our pruposes 
                        #than doc2vec embedding)
                        #v = TfidfVectorizer()
                        #response = v.fit_transform([billText])
                    except Exception:
                        print('Could not retrieve text for %s'%state_link)

NameError: name 'bloblist' is not defined

In [5]:
with open('NHlegtext.corpus','wb') as f:
    pickle.dump(legtext_corpus[1:], f)

In [2]:
print(rawbillText)

NameError: name 'rawbillText' is not defined

In [5]:
with open('NHlegtext.corpus','rb') as f:
    leg_corp = pickle.load(f)
    

In [8]:
print(leg_corp[0:3])

[['894704', '1441765', ['concurrent', 'resolution', 'proposing', 'consitutional', 'amendment', 'relating', 'general', 'court', 'providing', 'general', 'court', 'shall', 'hold', 'sessions', 'biennially', 'resolved', 'house', 'representatives', 'senate', 'concurring', 'constitution', 'new', 'hampshire', 'amended', 'follows', 'article', 'second', 'part', 'constitution', 'amended', 'read', 'follows', 'art', 'general', 'court', 'meet', 'dissolve', 'senate', 'house', 'shall', 'assemble', 'biennially', 'first', 'wednesday', 'december', 'organizational', 'purposes', 'even', 'numbered', 'years', 'following', 'organization', 'shall', 'assemble', 'annually', 'first', 'wednesday', 'following', 'first', 'tuesday', 'january', 'times', 'may', 'judge', 'necessary', 'shall', 'dissolve', 'dissolved', 'first', 'wednesday', 'december', 'even', 'numbered', 'years', 'biennially', 'shall', 'styled', 'general', 'court', 'new', 'hampshire', 'ii', 'amendment', 'proposed', 'constitution', 'submitted', 'qualified

In [9]:
print(pickle.load(state_link))

NameError: name 'state_link' is not defined